In [6]:
''' Install necessary packages '''

%pip install numpy
%pip install pandas
%pip install matplotlib
%pip install scikit-learn
%pip install pyarrow
%pip install fastparquet

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor




[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip3 install --upgrade pip
Note

In [2]:
''' Load the data '''

datapath = '../../IceCube_Training_Data/'

batch_list = [pd.read_parquet(datapath+'batch_'+str(i)+'.parquet') for i in range(1,6)]
rawdata = pd.concat(batch_list)
events = rawdata.index.drop_duplicates()
print(events)
print(rawdata.head())

Index([       24,        41,        59,        67,        72,        77,
              79,        82,       121,       127,
       ...
       331932694, 331932700, 331932728, 331932770, 331932779, 331932801,
       331932836, 331932852, 331932854, 331932899],
      dtype='int64', name='event_id', length=1000000)
          sensor_id  time  charge  auxiliary
event_id                                    
24             3918  5928   1.325       True
24             4157  6115   1.175       True
24             3520  6492   0.925       True
24             5041  6665   0.225       True
24             2948  8054   1.575       True


In [3]:

rawdata_meta = pd.read_parquet('../../train_meta.parquet')
rawdata_meta = rawdata_meta.loc[rawdata_meta['event_id'].isin(events)]
print(rawdata_meta.head())


   batch_id  event_id  first_pulse_index  last_pulse_index   azimuth    zenith
0         1        24                  0                60  5.029555  2.087498
1         1        41                 61               111  0.417742  1.549686
2         1        59                112               147  1.160466  2.401942
3         1        67                148               289  5.845952  0.759054
4         1        72                290               351  0.653719  0.939117


In [4]:

merged_data = pd.merge(rawdata, rawdata_meta, on='event_id')
print(merged_data.head())


   event_id  sensor_id  time  charge  auxiliary  batch_id  first_pulse_index   
0        24       3918  5928   1.325       True         1                  0  \
1        24       4157  6115   1.175       True         1                  0   
2        24       3520  6492   0.925       True         1                  0   
3        24       5041  6665   0.225       True         1                  0   
4        24       2948  8054   1.575       True         1                  0   

   last_pulse_index   azimuth    zenith  
0                60  5.029555  2.087498  
1                60  5.029555  2.087498  
2                60  5.029555  2.087498  
3                60  5.029555  2.087498  
4                60  5.029555  2.087498  


In [7]:
# Aggregate the data by 'event_id'
aggregated_data = merged_data.groupby('event_id').agg({
    'zenith': 'mean',
    'azimuth': 'mean',
    'sensor_id': 'count',   # number of sensors triggered
    'time': ['min', 'max'],  # earliest and latest trigger time
    'charge': ['min', 'max', 'mean']  # minimum, maximum, and average charge
})

# Flatten the column names
aggregated_data.columns = ['_'.join(col).strip() for col in aggregated_data.columns.values]


print(aggregated_data.head())


          zenith_mean  azimuth_mean  sensor_id_count  time_min  time_max   
event_id                                                                   
24           2.087498      5.029555               61      5928     19031  \
41           1.549686      0.417742               51      6469     17230   
59           2.401942      1.160466               36      6601     15362   
67           0.759054      5.845952              142      6751     27144   
72           0.939117      0.653719               62      6083     17654   

          charge_min  charge_max  charge_mean  
event_id                                       
24             0.175       3.975     0.991393  
41             0.225       1.875     0.907353  
59             0.175       2.625     0.922222  
67             0.125       5.125     0.927817  
72             0.125       1.675     0.899194  


In [27]:
''' Starting with the basics '''

X = aggregated_data.drop(['zenith_mean', 'azimuth_mean'], axis=1)
y = aggregated_data[['zenith_mean', 'azimuth_mean']]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, test_size=0.2, random_state=117)

# Train the multi-output random forest regressor

# We test multiple estimator values of the random forest and plot the score

predictions = []
estimator_number = np.arange(10,5100,500)
for i in range(len(estimator_number)):
    print('Starting model '+str(i+1)+' of '+str(len(estimator_number)))
    rf = RandomForestRegressor(n_estimators=estimator_number[i], random_state=117, verbose=1)
    mor = MultiOutputRegressor(rf)
    mor.fit(X_train, y_train)
    predictions.append(mor.predict(X_test))





Starting model 0 of 11


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   57.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  1.4min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    1.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    1.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Starting model 1 of 11


KeyboardInterrupt: 

In [ ]:


def angular_dist_score(az_true, zen_true, az_pred, zen_pred):
    '''
    calculate the MAE of the angular distance between two directions.
    The two vectors are first converted to cartesian unit vectors,
    and then their scalar product is computed, which is equal to
    the cosine of the angle between the two vectors. The inverse 
    cosine (arccos) thereof is then the angle between the two input vectors
    
    Parameters:
    -----------
    
    az_true : float (or array thereof)
        true azimuth value(s) in radian
    zen_true : float (or array thereof)
        true zenith value(s) in radian
    az_pred : float (or array thereof)
        predicted azimuth value(s) in radian
    zen_pred : float (or array thereof)
        predicted zenith value(s) in radian
    
    Returns:
    --------
    
    dist : float
        mean over the angular distance(s) in radian
    '''
    
    if not (np.all(np.isfinite(az_true)) and
            np.all(np.isfinite(zen_true)) and
            np.all(np.isfinite(az_pred)) and
            np.all(np.isfinite(zen_pred))):
        raise ValueError("All arguments must be finite")
    
    # pre-compute all sine and cosine values
    sa1 = np.sin(az_true)
    ca1 = np.cos(az_true)
    sz1 = np.sin(zen_true)
    cz1 = np.cos(zen_true)
    
    sa2 = np.sin(az_pred)
    ca2 = np.cos(az_pred)
    sz2 = np.sin(zen_pred)
    cz2 = np.cos(zen_pred)
    
    # scalar product of the two cartesian vectors (x = sz*ca, y = sz*sa, z = cz)
    scalar_prod = sz1*sz2*(ca1*ca2 + sa1*sa2) + (cz1*cz2)
    
    # scalar product of two unit vectors is always between -1 and 1, this is against nummerical instability
    # that might otherwise occure from the finite precision of the sine and cosine functions
    scalar_prod =  np.clip(scalar_prod, -1, 1)
    
    # convert back to an angle (in radian)
    return np.average(np.abs(np.arccos(scalar_prod)))


NameError: name 'pd' is not defined